# 1. Problem Statement:

Given labeled data (feature, target), what is an optimal supervised machine learning model that generalizes best out of sample.

Using grid search, a near optimal solution to the statement above can be quickly identified. By specifying our data, a model type, hyperparameter options, and an error metric, grid search uses cross validation to estimate the model's out of sample performance under different hyperparameter conditions enabling the selection of the combination that performs best i.e. minimizes our error metric of choice out of sample. By repeating this procedure for various models and associated hyperparameter options, we can identify a singular model type and hyperparameter settings that are estimated to perform best on new data.

**Variants:**
- Grid search methods can be varied across a variety of approaches: traditional, telescopic, random search, etc.
- Threshold analysis can also be incorporated. In addition to returning a hard class assignment in classification settings, many models can also return a probabilistic output that corresponds to the P(Y = class k | x) for each class. In binary classification settings (Y = +/- 1), a threshold of 0.5 is typically assumed where if P(Y = class 1 | x) >= 0.5, predict 1 else predict -1. By storing out of sample probabilistic predictions throughout the cross validation process, we are also able to inspect whether modifying a model's probability threshold allows us to optimize performance even further with respect to our error metric of choice.

**Key assumptions:**  

This framework is primarily optimized for evaluating out of sample performance of classical machine learning models rather than deep learning models due to training time, lack of sensitivity to starting points for parametric models, and neural architecture training stoppage largely being dependent on validation set performance which is measured after each training epoch. While it can be used for deep learning, expect longer execution time.

# 2. Develop grid search pipeline for IDing optimal hyperparameter settings given labeled data and model type

### Import Packages and Libraries

In [1]:
# Data Preparation
import pandas as pd
import numpy as np

#ML Models
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.naive_bayes import CategoricalNB, GaussianNB

#ML Model Evaluation
from sklearn.metrics import (accuracy_score,f1_score,precision_score,
                             recall_score,roc_auc_score,
                             mean_squared_error,mean_absolute_error,r2_score)

#Grid Search Methods
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,KFold

#Clone Model
from sklearn.base import clone

#Parallel processing
import multiprocessing

#Dummy Dataset Generation
from sklearn.datasets import make_classification,make_regression

### Develop Pipeline

#### General Scoring Function to evaluate model performance given true, pred values -- alter as needed

In [2]:
def scoring_function(ytrue,ypred,metric):
    if metric == 'f1':
        score = f1_score(ytrue,ypred)
    if metric == 'accuracy':
        score = accuracy_score(ytrue,ypred)
    if metric == 'precision':
        score = precision_score(ytrue,ypred)
    if metric == 'recall':
        score = recall_score(ytrue,ypred)
    if metric == 'neg_mean_absolute_error':
        score = mean_absolute_error(ytrue,ypred)
    if metric == 'neg_mean_squared_error':
        score = mean_squared_error(ytrue,ypred)
    if metric == 'r2':
        score = r2_score(ytrue,ypred)
    if metric == 'roc_auc':
        score = roc_auc_score(ytrue,ypred)
    
    return score

#### Optimal model development pipeline -- given labeled data and model type, identify model hyperparameters that optimize out of sample parameters therefore maximizing estimated generalization capability

In [3]:
def optimal_model_dev(X,y,model_type,param_options,metric,metric_direction,
                      search_method = 'trad',num_folds = 5,threshold_analysis_bool=False,random_state=50):
    '''
    X: features, assumes pandas df/series
    y: labels, assumes pandas df/series
    model_type: type of ML model
    param_options: hyperparameter options for the given model to be evaluated through parameter search process
    metric: metric we are trying to optimize out of sample, accepts one of ['accuracy','precision','recall','f1','roc_auc','neg_mean_squared_error','neg_mean_absolute_error','r2']
    metric_direction: 'max' if higher metric = better, 'min' if lower metric = better
    search_method: 'trad' = traditional grid search, 'rand' = random search
    num_folds: number of folds throughout the cross validation process internal to grid search
    threshold_analysis_bool: FOR BINARY CLASSIFICATION ONLY -- True to conduct out of sample threshold analysis, False otherwise
    random_state: random state for reproducable results
    '''
    #Dictionary to Store Optimal Model Results
    return_dict = {}
    
    #Create copies to avoid broadcasting
    X_copy = X.copy()
    y_copy = y.copy()
    
    #Instantiate search method/search space
    if search_method == 'trad':
        grid_search = GridSearchCV(estimator=model_type,param_grid=param_options,scoring=metric,
                                   n_jobs=multiprocessing.cpu_count()-1,cv=num_folds)
    elif search_method == 'rand':
        #set number of combinations to evaluate
        possible_combos = np.prod([len(x) for x in param_options.values()])
        if possible_combos > 10 and possible_combos < 30:
            possible_combos = 10
        elif possible_combos >= 30:
            possible_combos = int((1/3)*possible_combos)
            
        grid_search = RandomizedSearchCV(estimator=model_type,param_distributions=param_options,scoring=metric,
                                        n_jobs=multiprocessing.cpu_count()-1,cv=num_folds,
                                        n_iter=possible_combos)
    
    
    #Fit search method
    grid_search.fit(X_copy,y_copy)
    
    #store results of best performing model out of sample --> best hyperparameter settings to optimize model performance
    return_dict['best_model'] = grid_search.best_estimator_
    return_dict['best_params'] = grid_search.best_params_
    return_dict['metric'] = metric
    return_dict['best_score'] = grid_search.best_score_
    
    #conduct threshold analysis in binary class setting to find probabilistic threshold that optimizes performance
    #of currently identified best performing model out of sample
    if threshold_analysis_bool and y_copy.nunique() == 2:
        oos_preds = y.copy() #store preds out of sample in CV process
        kfold = KFold(n_splits=num_folds)#instantiate kfold object
        model = clone(grid_search.best_estimator_)#clone model to capture best model's hyperparameters w/o training
        
        for train,test in kfold.split(X_copy):
            #split data on k-1 folds for train, 1 test fold
            xtrain,xtest,ytrain,ytest = X_copy.iloc[train],X_copy.iloc[test],y_copy.iloc[train],y_copy.iloc[test]
            model.fit(xtrain,ytrain) #fit model
            oos_preds.iloc[test] = list(model.predict_proba(xtest)[:,1]) #make preds, store out of sample pred P(Y = 1 | X)
        
        #conduct threshold analysis
        best_score = grid_search.best_score_
        best_thresh = 0.5
        for num in np.linspace(0.01,0.999,500):
            mapped_preds = oos_preds.apply(lambda x:1 if x>num else 0) #map probs to preds according to threshold
            #evaluate performance
            score = scoring_function(y,mapped_preds,metric)
            if (metric_direction == 'max' and score>best_score) or (metric_direction == 'min' and score < best_score):
                best_score = score
                best_thresh = num
        
        return_dict['best_thresh'] = best_thresh
        return_dict['best_score_adjusted'] = best_score
            
    return return_dict
    

### Test Optimal Model Development Pipeline

#### Create Dummy Data

In [4]:
#Binary Classification
binary_classification_data = make_classification(n_samples=2000,n_features=5,n_informative=5,n_redundant=0,n_classes=2)

#Multiclass classification
multiclass_classification_data = make_classification(n_samples=4000,n_features=5,n_informative=5,n_redundant=0,n_classes=4)

#Regression
regression_data = make_regression(n_samples = 1000, n_features = 5,n_informative=5)

#### Binary Classification Test

In [5]:
features = pd.DataFrame(binary_classification_data[0])
label = pd.Series(binary_classification_data[1])
label.value_counts()/len(label)

0    0.501
1    0.499
dtype: float64

In [6]:
models = [KNeighborsClassifier(),RandomForestClassifier(random_state=50),
         GradientBoostingClassifier(max_features='sqrt'),GaussianNB(),LogisticRegression(solver='saga')]
params = [{'n_neighbors':[3,5,7,9,11,13,15,17,19],'p':[1,2]},
         {'n_estimators':[100,200,300,400],'criterion':['gini','entropy'],'min_samples_split':[2,4,6]},
         {'n_estimators':[100,200,300,400],'max_depth':[2,3,4,6,8]},
         {'var_smoothing':[0.0000001,0.000001,0.00001,0.0001,0.001,0.01,0.1,1]},
         {'C':[0.00001,0.0001,0.001,0.01,0.1,1,10],'penalty':['l1','l2']}
         ]

best_results = ''
best_model = ''
best_score = 0
metric = 'accuracy'
metric_direction = 'max'
for model,param in zip(models,params):
    results = optimal_model_dev(features[:1600],label[:1600],model,param,
                      metric,metric_direction,'rand',5,True,50)
    if results['best_score_adjusted'] > best_score:
        best_results = results
        best_score = results['best_score_adjusted']
        best_model = results['best_model']
    print(str(results) + '\n')

print('Final Evaluation')
print(best_model)
print(scoring_function(label[1600:],
                 [1 if x >= best_results['best_thresh'] else 0 for x in best_model.predict_proba(features[1600:])[:,1]],
                 metric=metric))

{'best_model': KNeighborsClassifier(n_neighbors=7), 'best_params': {'p': 2, 'n_neighbors': 7}, 'metric': 'accuracy', 'best_score': 0.929375, 'best_thresh': 0.4301763527054108, 'best_score_adjusted': 0.93}

{'best_model': RandomForestClassifier(criterion='entropy', n_estimators=300, random_state=50), 'best_params': {'n_estimators': 300, 'min_samples_split': 2, 'criterion': 'entropy'}, 'metric': 'accuracy', 'best_score': 0.9174999999999999, 'best_thresh': 0.531256513026052, 'best_score_adjusted': 0.918125}

{'best_model': GradientBoostingClassifier(max_depth=6, max_features='sqrt', n_estimators=300), 'best_params': {'n_estimators': 300, 'max_depth': 6}, 'metric': 'accuracy', 'best_score': 0.9200000000000002, 'best_thresh': 0.24387174348697394, 'best_score_adjusted': 0.92125}

{'best_model': GaussianNB(var_smoothing=1), 'best_params': {'var_smoothing': 1}, 'metric': 'accuracy', 'best_score': 0.8231249999999999, 'best_thresh': 0.5035090180360721, 'best_score_adjusted': 0.826875}

{'best_mo

#### Multiclass Classification test

In [7]:
features = pd.DataFrame(multiclass_classification_data[0])
label = pd.Series(multiclass_classification_data[1])
label.value_counts()/len(label)

2    0.25100
0    0.25000
3    0.24975
1    0.24925
dtype: float64

In [8]:
models = [KNeighborsClassifier(),RandomForestClassifier(random_state=50),
         GradientBoostingClassifier(max_features='sqrt'),GaussianNB(),LogisticRegression(solver='saga')]
params = [{'n_neighbors':[3,5,7,9,11,13,15,17,19],'p':[1,2]},
         {'n_estimators':[100,200,300,400],'criterion':['gini','entropy'],'min_samples_split':[2,4,6]},
         {'n_estimators':[100,200,300,400],'max_depth':[2,3,4,6,8]},
         {'var_smoothing':[0.0000001,0.000001,0.00001,0.0001,0.001,0.01,0.1,1]},
         {'C':[0.00001,0.0001,0.001,0.01,0.1,1,10],'penalty':['l1','l2']}
         ]

best_results = ''
best_model = ''
best_score = 0
metric = 'accuracy'
metric_direction = 'max'
for model,param in zip(models,params):
    results = optimal_model_dev(features[:3200],label[:3200],model,param,
                      metric,metric_direction,'rand',5,True,50)
    if results['best_score'] > best_score:
        best_results = results
        best_score = results['best_score']
        best_model = results['best_model']
    print(str(results) + '\n')

print('Final Evaluation')
print(best_model)
print(scoring_function(label[3200:],best_model.predict(features[3200:]),
                 metric=metric))

{'best_model': KNeighborsClassifier(n_neighbors=9), 'best_params': {'p': 2, 'n_neighbors': 9}, 'metric': 'accuracy', 'best_score': 0.8306250000000001}

{'best_model': RandomForestClassifier(criterion='entropy', n_estimators=300, random_state=50), 'best_params': {'n_estimators': 300, 'min_samples_split': 2, 'criterion': 'entropy'}, 'metric': 'accuracy', 'best_score': 0.81125}

{'best_model': GradientBoostingClassifier(max_depth=8, max_features='sqrt', n_estimators=200), 'best_params': {'n_estimators': 200, 'max_depth': 8}, 'metric': 'accuracy', 'best_score': 0.8146875}

{'best_model': GaussianNB(var_smoothing=0.1), 'best_params': {'var_smoothing': 0.1}, 'metric': 'accuracy', 'best_score': 0.645625}

{'best_model': LogisticRegression(C=0.1, penalty='l1', solver='saga'), 'best_params': {'penalty': 'l1', 'C': 0.1}, 'metric': 'accuracy', 'best_score': 0.6353125000000001}

Final Evaluation
KNeighborsClassifier(n_neighbors=9)
0.8325


/Users/Amit.Gattadahallic3.ai/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/Amit.Gattadahallic3.ai/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/Amit.Gattadahallic3.ai/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/Amit.Gattadahallic3.ai/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/Amit.Gatt

#### Regression Test

In [9]:
features = pd.DataFrame(regression_data[0])
label = pd.Series(regression_data[1])

In [10]:
models = [KNeighborsRegressor(),RandomForestRegressor(random_state=50),
         GradientBoostingRegressor(max_features='sqrt')]
params = [{'n_neighbors':[3,5,7,9,11,13,15,17,19],'p':[1,2]},
         {'n_estimators':[100,200,300,400],'min_samples_split':[2,4,6]},
         {'n_estimators':[100,200,300,400],'max_depth':[2,3,4,6,8]}
         ]

best_results = ''
best_model = ''
best_score = 0
metric = 'r2'
metric_direction = 'max'
for model,param in zip(models,params):
    results = optimal_model_dev(features[:800],label[:800],model,param,
                      metric,metric_direction,'rand',5,True,50)
    if results['best_score'] > best_score:
        best_results = results
        best_score = results['best_score']
        best_model = results['best_model']
    print(str(results) + '\n')

print('Final Evaluation')
print(best_model)
print(scoring_function(label[800:],best_model.predict(features[800:]),
                 metric=metric))

{'best_model': KNeighborsRegressor(n_neighbors=3), 'best_params': {'p': 2, 'n_neighbors': 3}, 'metric': 'r2', 'best_score': 0.9109167407182271}

{'best_model': RandomForestRegressor(n_estimators=200, random_state=50), 'best_params': {'n_estimators': 200, 'min_samples_split': 2}, 'metric': 'r2', 'best_score': 0.9195511031482153}

{'best_model': GradientBoostingRegressor(max_depth=2, max_features='sqrt', n_estimators=400), 'best_params': {'n_estimators': 400, 'max_depth': 2}, 'metric': 'r2', 'best_score': 0.9849280706856236}

Final Evaluation
GradientBoostingRegressor(max_depth=2, max_features='sqrt', n_estimators=400)
0.9853906588458496
